In [38]:
import re
import numpy as np
import pandas as pd
from typing import List, Optional, Callable, Dict
pd.set_option('display.max_columns', None) 

In [39]:
dfs = {}
for year in range(2020, 2025):
    for type in ['전기승용','전기화물']:
        year_str = str(year)
        filename = f'./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_{type}/구매보조금_전기차_{type}_{year}.xls'
        
        try:
            df = pd.read_excel(filename,  header=2)
            dfs[(year, type)] = df[['시도','지역구분','차종','민간공고대수','출고대수','출고잔여대수']]
            print(f'{filename} 불러오기 성공')
        except FileNotFoundError:
            print(f'{filename} 파일이 존재하지 않음')
        except Exception as e:
            print(f'{filename} 오류 발생: {e}')

./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_전기승용/구매보조금_전기차_전기승용_2020.xls 불러오기 성공
./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_전기화물/구매보조금_전기차_전기화물_2020.xls 파일이 존재하지 않음
./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_전기승용/구매보조금_전기차_전기승용_2021.xls 불러오기 성공
./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_전기화물/구매보조금_전기차_전기화물_2021.xls 불러오기 성공
./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_전기승용/구매보조금_전기차_전기승용_2022.xls 불러오기 성공
./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_전기화물/구매보조금_전기차_전기화물_2022.xls 불러오기 성공
./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_전기승용/구매보조금_전기차_전기승용_2023.xls 불러오기 성공
./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_전기화물/구매보조금_전기차_전기화물_2023.xls 불러오기 성공
./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_전기승용/구매보조금_전기차_전기승용_2024.xls 불러오기 성공
./01_raw_data/EV_project/정책·지원/구매보조금 지급대수(시군구)/구매보조금_전기차_전기화물/구매보조금_전기차_전기화물_2024.xls 불러오기 성공


In [50]:
for year in range(2020, 2025):
    for ev_type in ['전기승용','전기화물']:
        if (year == 2020) and (ev_type == '전기화물'):
            break  
        dfs[(year, ev_type)] = dfs[(year, ev_type)].applymap(
             lambda x: str(x).replace("(", "").replace(")", "") if pd.notna(x) else x
        )
dfs[(2020,'전기승용')]

C:\Users\junjo\AppData\Local\Temp\ipykernel_13876\2964396347.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfs[(year, ev_type)] = dfs[(year, ev_type)].applymap(


,시도,지역구분,차종,민간공고대수,출고대수,출고잔여대수
0,NaN,NaN,NaN,전체\n우선순위\n일반,전체\n우선순위\n일반,전체\n우선순위\n일반
1,협회,한국자동차환경협회,화물,1000\n 0\n 1000,845\n 4\n 841,155\n 0\n 155
2,서울,서울특별시,승용,5132\n 0\n 5132,5551\n 211\n 5340,0\n 0\n 0
3,서울,서울특별시,화물,1271\n 0\n 1271,1288\n 25\n 1263,0\n 0\n 0
4,서울,서울특별시,승합,119\n 0\n 119,249\n 0\n 249,0\n 0\n 0
...,...,...,...,...,...,...
345,NaN,NaN,NaN,NaN,NaN,NaN
346,NaN,NaN,NaN,NaN,NaN,NaN
347,NaN,NaN,NaN,NaN,NaN,NaN
348,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def split_by_newline(cell: object, preprocess: Optional[Callable[[str], str]] = None) -> List[str]:
    """셀 문자열을 \\n 기준으로 분리해 trim한 리스트를 반환"""
    if pd.isna(cell):
        return []
    s = str(cell).replace("\r", "\n")
    if preprocess:
        s = preprocess(s)
    parts = [p.strip() for p in s.split("\n")]
    return [p for p in parts if p != ""]

def looks_like_header(values: List[str]) -> bool:
    """
    값 리스트가 헤더(문자 포함)처럼 보이는지 판정:
    - 길이 >= 2
    - 모든 원소가 숫자(정수/실수)만은 아님
    """
    if len(values) < 2:
        return False
    only_numbers = all(bool(re.fullmatch(r"-?\d+(?:\.\d+)?", v)) for v in values)
    return not only_numbers

def expand_newline_columns(
    df: pd.DataFrame,
    columns_to_expand: List[str],
    id_cols: Optional[List[str]] = None,
    preprocess: Optional[Callable[[str], str]] = None,
    to_numeric: bool = True,
) -> pd.DataFrame:

    if id_cols is None:
        candidates = ["시도", "지역구분", "차종"]
        id_cols = [c for c in candidates if c in df.columns]

    out = df.copy()
    header_row_mask = None
    if id_cols:
        header_row_mask = out[id_cols].isna().all(axis=1)
    rows_to_drop = set()

    for col in columns_to_expand:
        if col not in out.columns:
            raise KeyError(f"'{col}' 컬럼이 없습니다.")

        lists = out[col].apply(lambda x: split_by_newline(x, preprocess=preprocess))
        header_idx = None
        if header_row_mask is not None and header_row_mask.any():
            for idx, lst in lists.items():
                if header_row_mask.loc[idx] and looks_like_header(lst):
                    header_idx = idx
                    header_values = lst
                    break
        else:
            for idx, lst in lists.items():
                if looks_like_header(lst):
                    header_idx = idx
                    header_values = lst
                    break

        if header_idx is not None:
            cats = header_values
            rows_to_drop.add(header_idx)
            max_len = len(cats)
            col_names = [f"{col}_{c}" for c in cats]
        else:
            max_len = lists.map(len).max() or 0
            col_names = [f"{col}_{i+1}" for i in range(max_len)]

        def pad(xs: List[str]) -> List[Optional[str]]:
            return (xs + [None] * (max_len - len(xs)))[:max_len]

        expanded = pd.DataFrame(lists.apply(pad).tolist(), index=out.index, columns=col_names)

        out = pd.concat([out.drop(columns=[col]), expanded], axis=1)

        if to_numeric:
            for c in col_names:
                out[c] = pd.to_numeric(out[c], errors='coerce')

    if rows_to_drop:
        out = out.drop(index=list(rows_to_drop))

    out = out.reset_index(drop=True)

    front = [c for c in (id_cols or []) if c in out.columns]
    rest = [c for c in out.columns if c not in front]
    return out[front + rest] if front else out

dfs2= {}

for year in range(2020, 2025):
    for ev_type in ['전기승용','전기화물']:
        if (year == 2020) and (ev_type == '전기화물'):
            break  
        dfs2[(year, ev_type)] = expand_newline_columns(dfs[(year, ev_type)], ["민간공고대수", "출고대수", "출고잔여대수"],
                              id_cols=["시도","지역구분","차종"])
        
        


,시도,지역구분,차종,민간공고대수_전체,민간공고대수_우선순위,민간공고대수_중소기업,민간공고대수_법인·기관,민간공고대수_일반,출고대수_전체,출고대수_우선순위,출고대수_중소기업,출고대수_법인·기관,출고대수_택배,출고대수_일반,출고잔여대수_전체,출고잔여대수_우선순위,출고잔여대수_중소기업,출고잔여대수_법인·기관,출고잔여대수_택배,출고잔여대수_일반
0,서울,서울특별시,전기화물,2495.0,395.0,200.0,0.0,0.0,2501.0,158.0,325.0,0.0,0.0,2018.0,0.0,237.0,0.0,0.0,0.0,0.0
1,부산,부산광역시,전기화물,1239.0,16.0,57.0,0.0,0.0,1382.0,14.0,53.0,0.0,0.0,1315.0,0.0,2.0,4.0,0.0,0.0,0.0
2,대구,대구광역시,전기화물,1248.0,0.0,0.0,0.0,0.0,1232.0,1.0,44.0,0.0,0.0,1187.0,16.0,0.0,0.0,0.0,0.0,61.0
3,인천,인천광역시,전기화물,1323.0,0.0,0.0,0.0,0.0,1305.0,19.0,52.0,0.0,0.0,1234.0,18.0,0.0,0.0,0.0,0.0,89.0
4,광주,광주광역시,전기화물,778.0,78.0,78.0,0.0,0.0,769.0,20.0,22.0,0.0,0.0,727.0,9.0,58.0,56.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_df_list = []
for (year, ev_type), d in dfs2.items():
    dd = d.copy()
    dd["년도"] = year
    all_df_list.append(dd)

final_df = pd.concat(all_df_list, ignore_index=True)

if "차종" in final_df.columns:
    final_df = final_df[~final_df["차종"].isin(["전체", "승합"])].copy()
keys = [k for k in ["년도", "시도", "차종"] if k in final_df.columns]

tmp = final_df.copy()
value_cols = [c for c in tmp.columns if c not in keys]
tmp[value_cols] = tmp[value_cols].apply(pd.to_numeric, errors="coerce")

merged_df = (
    tmp.groupby(keys, as_index=False)
       .sum(numeric_only=True)
       .fillna(0)
)

merged_df.head()

,년도,시도,차종,지역구분,민간공고대수_전체,민간공고대수_우선순위,민간공고대수_일반,출고대수_전체,출고대수_우선순위,출고대수_일반,출고잔여대수_전체,출고잔여대수_우선순위,출고잔여대수_일반,민간공고대수_법인·기관,민간공고대수_택시,출고대수_법인·기관,출고대수_택시,출고잔여대수_법인·기관,출고잔여대수_택시,민간공고대수_중소기업,출고대수_중소기업,출고대수_택배,출고잔여대수_중소기업,출고잔여대수_택배
0,2020,강원,승용,0.0,1252.0,12.0,1240.0,869.0,33.0,836.0,384.0,10.0,382.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020,강원,화물,0.0,412.0,8.0,404.0,720.0,15.0,705.0,23.0,4.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020,경기,승용,0.0,5013.0,210.0,4803.0,5122.0,494.0,4628.0,173.0,73.0,173.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020,경기,화물,0.0,1795.0,30.0,1765.0,1747.0,156.0,1591.0,222.0,3.0,222.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020,경남,승용,0.0,1572.0,9.0,1563.0,1532.0,89.0,1443.0,61.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
df_copy = merged_df.copy()

df_copy["차종"] = df_copy["차종"].replace({
    "전기승용": "승용",
    "전기화물": "화물"
})

select_df = df_copy[['년도','시도','차종','민간공고대수_전체','출고대수_전체','출고잔여대수_전체']]
select_df = select_df.rename(columns={
    '시도': '지역',
    '민간공고대수_전체':'민간공고대수',
    '출고대수_전체':'출고대수' ,	
    '출고잔여대수_전체':'출고잔여대수'
    })
select_df

,년도,지역,차종,민간공고대수,출고대수,출고잔여대수
0,2020,강원,승용,1252.0,869.0,384.0
1,2020,강원,화물,412.0,720.0,23.0
2,2020,경기,승용,5013.0,5122.0,173.0
3,2020,경기,화물,1795.0,1747.0,222.0
4,2020,경남,승용,1572.0,1532.0,61.0
...,...,...,...,...,...,...
169,2024,제주,화물,2300.0,1150.0,1150.0
170,2024,충남,승용,4131.0,4433.0,143.0
171,2024,충남,화물,2672.0,2141.0,670.0
172,2024,충북,승용,4888.0,4729.0,544.0


In [44]:
select_df.to_csv('./02_interim_data/subsidy_count.csv', index=False, encoding='utf-8-sig')

In [45]:
# df_copy = merged_df.copy()

# mask = (df_copy["년도"] == 2020) & (df_copy["차종"] == "승합")

# src_cols_all = [c for c in df_copy.columns if c.startswith("전기승용_")]
# pairs = []
# for s in src_cols_all:
#     t = s.replace("전기승용_", "전기화물_")
#     if t in df_copy.columns:
#         pairs.append((s, t))

# src_cols = [s for s, _ in pairs]
# tgt_cols = [t for _, t in pairs]

# df_copy.loc[mask, tgt_cols] = df_copy.loc[mask, src_cols].to_numpy()
# df_copy.loc[mask, src_cols] = 0  # 원본 승용 값 초기화
# df_copy["차종"] = df_copy["차종"].replace({
#     "전기승용": "승용",
#     "전기화물": "화물"
# })
# df_copy[df_copy['년도']==2020]